# Agent S: Nuclei Segmentation

This notebook segments nuclei from the DAPI channel of the MIP images generated in Step 1. It utilizes Cellpose for initial segmentation and optionally SAM for refinement.

### Goals:
1. Load MIP images (DAPI channel).
2. Perform segmentation with user-tuned parameters (Diameter, Flow Threshold).
3. Generate unique Nucleus IDs.
4. Export masks and metadata (Pickle) for spot assignment.

In [ ]:
import os
import pickle
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models
from skimage import measure

# Default rendering settings
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 10)

## 1. Parameters

Adjust the following parameters for your dataset. The DAPI channel is usually the last channel in our compiled MIPs.

In [ ]:
mip_dir = "./processed_data/01_MIPs"
output_dir = "./processed_data/02_Segmentation"
dapi_channel_idx = -1  # Assuming DAPI is the last channel

# Cellpose Parameters
nucleus_diameter = 80.0
flow_threshold = 0.4
cellprob_threshold = 0.0

os.makedirs(output_dir, exist_ok=True)

## 2. Load Model

Initializing the Cellpose 'nuclei' model.

In [ ]:
model = models.Cellpose(gpu=True, model_type='nuclei')

## 3. Segment and Save

Processing all MIPs in the directory.

In [ ]:
mip_files = [f for f in os.listdir(mip_dir) if f.endswith(".tif")]
print(f"Found {len(mip_files)} MIP files.")

for filename in mip_files:
    path = os.path.join(mip_dir, filename)
    img = tifffile.imread(path)
    
    # Extract DAPI channel
    dapi = img[dapi_channel_idx] if img.ndim == 3 else img
    
    print(f"Segmenting {filename}...")
    masks, flows, styles, diams = model.eval(
        dapi,
        diameter=nucleus_diameter,
        channels=[0,0],
        flow_threshold=flow_threshold,
        cellprob_threshold=cellprob_threshold
    )
    
    # Save Mask TIFF
    mask_path = os.path.join(output_dir, filename.replace(".tif", "_masks.tif"))
    tifffile.imwrite(mask_path, masks.astype(np.uint16))
    
    # Extract Nucleus Data (compatible with GGscripts)
    props = measure.regionprops(masks)
    nucleus_data = []
    for p in props:
        nucleus_data.append({
            'id': p.label,
            'centroid': p.centroid,
            'bbox': p.bbox,
            'area': p.area
        })
    
    # Save Metadata Pickle
    pkl_data = {
        'filename': filename,
        'masks_nuclei': masks,
        'nucleus_data': nucleus_data,
        'params': {
            'diameter': nucleus_diameter,
            'flow_threshold': flow_threshold
        }
    }
    
    pkl_path = os.path.join(output_dir, filename.replace(".tif", "_segmentation.pkl"))
    with open(pkl_path, 'wb') as f:
        pickle.dump(pkl_data, f)
    
    print(f"  Saved: {mask_path}")
    print(f"  Found {len(nucleus_data)} nuclei.")


## 4. Visualization

Overlay masks on DAPI for the last processed image.

In [ ]:
if 'masks' in locals():
    from cellpose import plot
    fig = plt.figure(figsize=(12,12))
    outline = plot.mask_overlay(dapi, masks)
    plt.imshow(outline)
    plt.title(f"Segmentation Overlay: {filename}")
    plt.axis('off')
    plt.show()